In [2]:
# Importing Libaries
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

df = pd.read_csv('Open Table Review pre processed.csv')
df.head()

,Name,Date,Cusine,Overall_rating,Food_rating,Service_rating,Ambience_Rating,Review,Location,Restaurant
0,JustinD,31/01/2024,"Japanese, International, Seafood",5,5,5,5,I want Andy as my server every time. Andy is t...,"#70 - 200 Granville St, Vancouver, BC V6C1S4",Miku Restaurant
1,CarolineF,29/01/2024,"Japanese, International, Seafood",5,5,5,5,Fantastic and delicious food. Talented chefs a...,"#70 - 200 Granville St, Vancouver, BC V6C1S4",Miku Restaurant
2,Abby,29/01/2024,"Japanese, International, Seafood",5,5,5,5,Fantastic food and atmosphere. Signature Shoka...,"#70 - 200 Granville St, Vancouver, BC V6C1S4",Miku Restaurant
3,OpenTable Diner,28/01/2024,"Japanese, International, Seafood",5,5,5,5,Absolutely the best sushi experience I have ev...,"#70 - 200 Granville St, Vancouver, BC V6C1S4",Miku Restaurant
4,OpenTable Diner,28/01/2024,"Japanese, International, Seafood",5,5,5,5,"Just the best sushi ever, anywhere. Never disa...","#70 - 200 Granville St, Vancouver, BC V6C1S4",Miku Restaurant


### Defining a aspect analyzing function that is used to get the aspects as a list and the dataframe and returns two dataframes with aspects and their Positive and Negative sentiment score, average as well as total and the number of mentions as well respectively 

In [7]:

def aspect_analyzer(df, aspects):
    sia = SentimentIntensityAnalyzer()
    # Initialize dictionaries to hold positive and negative aspect data
    Pos_aspect = {aspect: {} for aspect in aspects}
    Neg_aspect = {aspect: {} for aspect in aspects}
    
    for index, row in df.iterrows():
        review = row['Review']
        restaurant = row['Restaurant']
        for aspect in aspects:
            aspect_sentences = [sentence for sentence in sent_tokenize(review) if aspect in sentence]
            for sentence in aspect_sentences:
                sentiment_score = sia.polarity_scores(sentence)['compound']
                # Check for positive sentiment
                if sentiment_score > 0.3:
                    if restaurant not in Pos_aspect[aspect]:
                        Pos_aspect[aspect][restaurant] = {'score': 0, 'mentions': 0}
                    Pos_aspect[aspect][restaurant]['score'] += sentiment_score
                    Pos_aspect[aspect][restaurant]['mentions'] += 1
                # Check for negative sentiment
                elif sentiment_score < -0.3:
                    if restaurant not in Neg_aspect[aspect]:
                        Neg_aspect[aspect][restaurant] = {'score': 0, 'mentions': 0}
                    Neg_aspect[aspect][restaurant]['score'] += sentiment_score
                    Neg_aspect[aspect][restaurant]['mentions'] += 1
    
    # Calculate averages and prepare dataframes
    positive_data = []
    negative_data = []
    for aspect in aspects:
        for restaurant, data in Pos_aspect[aspect].items():
            data['average'] = data['score'] / data['mentions'] if data['mentions'] > 0 else 0
            positive_data.append({'Aspect': aspect, 'Restaurant': restaurant, **data})
        for restaurant, data in Neg_aspect[aspect].items():
            data['average'] = data['score'] / data['mentions'] if data['mentions'] > 0 else 0
            negative_data.append({'Aspect': aspect, 'Restaurant': restaurant, **data})
    
    positive_df = pd.DataFrame(positive_data)
    negative_df = pd.DataFrame(negative_data)
    
    return positive_df, negative_df

# Define the aspects
aspects = ['food', 'service', 'price', 'portion', 'music', 'drink', 'recommend', 'expensive', 'cheap', 'loud', 'staff', 'rude']

# Example usage
df['Review'] = df['Review'].astype(str)
Positive_aspect_sentiment, Negative_aspect_sentiment = aspect_analyzer(df, aspects)
print(Positive_aspect_sentiment)
print(Negative_aspect_sentiment)


    Aspect                                Restaurant    score  mentions  \
0     food                           Miku Restaurant  82.4615       119   
1     food                      LAbattoir Restaurant  95.8015       136   
2     food                               Kissa Tanto  88.5346       126   
3     food                             Fable Kitchen  75.0269       103   
4     food                   Homer Street Cafe & Bar  94.7921       135   
..     ...                                       ...      ...       ...   
115  staff  TFanny Bay Oyster Bar & Shellfish Market  22.7624        30   
116  staff                     Yuwa Japanese Cuisine  15.8739        25   
117  staff                                  Chang;an  11.9447        18   
118  staff                                 The Lobby  10.8709        16   
119   rude  TFanny Bay Oyster Bar & Shellfish Market   0.7269         1   

      average  
0    0.692954  
1    0.704423  
2    0.702656  
3    0.728417  
4    0.702164  
.. 

### Exporting the dataframe to excel sheet

In [9]:
Positive_aspect_sentiment.to_excel("C:/Users/12367/Documents/Python Scripts/Positive Aspect Analysis.xlsx", index=False)
Negative_aspect_sentiment.to_excel("C:/Users/12367/Documents/Python Scripts/Negative Aspect Analysis.xlsx", index=False)
